In [12]:
#input DATAFRAME [topic, query, tag]
import pandas as pd
import time
import requests

from collections import Counter
from itertools import chain

class ChatNoir:
    

    def __init__(self, queries: pd.DataFrame, size ):
        self.url = 'https://www.chatnoir.eu/api/v1/_search'
        self.all_queries = queries
        self.size = size
    
    def get_response(self):
        df = pd.DataFrame()
        df['topic'] = self.all_queries['topic']
        df['query'] = self.all_queries['query']
        df['tag'] = self.all_queries['tag']

        resp = []

        for i in range(0, len(self.all_queries.index)):
            query = self.all_queries.iloc[i]['query']

            request_data = {
            "apikey": "67fac2d9-0f98-4c19-aab0-18c848bfa130",
            "query": query,
            "size": self.size,
            "index": ["cw12"],
            }
            
            def chatnoir_req():
                try:
                    response_=requests.post(self.url, data=request_data)
                    response_.raise_for_status()
                    return response_.json()
                except requests.exceptions.HTTPError:
                    time.sleep(1)
                    return chatnoir_req()   
             
            result = chatnoir_req()
            filter_result = []
            for doc in result['results']:
                answer = {
                    'trec_id': doc['trec_id'], 
                    'uuid': doc['uuid'], 
                    'title': doc['title'], 
                    'snippet': doc['snippet'],
                    'target_hostname': doc['target_hostname'],
                    'score': doc['score']
                    }
                filter_result.append(answer)
            
            resp.append({'results': filter_result})
        
        df['response'] = resp
        
        return df


if __name__ == "__main__":

    topics = ['What is the difference between sex and love?',
              'Which is the highest mountain in the world?',
               'Which is better, a laptop or a desktop?'
               ]
    df = pd.DataFrame()
    df['topic'] = [
        'What is the difference between sex and love?',
        'What is the difference between sex and love?',
        'Which is the highest mountain in the world?',  
        'Which is the highest mountain in the world?',
        'Which is better, a laptop or a desktop?',
        'Which is better, a laptop or a desktop?'
    ]
    df['query'] = [
        'What is the difference between sex and love?',
        'difference between sex and love',
        'Which is the highest mountain in the world?',
        'highest mountain world',
        'Which is better, a laptop or a desktop?',
        'better laptop or desktop'
    ]
    df['tag'] = [
        'original',
        'annotation',
        'original',
        'annotation',
        'original',
        'annotation'
    ]
    pd.set_option('display.max_columns', 10)
    
    chatnoir = ChatNoir(df, size=5)
    print(10*"=" + "RESULTS FROM CHATNOIR" + 10*"=")
    resp_df = chatnoir.get_response()
    print(resp_df)

==========RESULTS FROM CHATNOIR==========
                                          topic  \
0  What is the difference between sex and love?   
1  What is the difference between sex and love?   
2   Which is the highest mountain in the world?   
3   Which is the highest mountain in the world?   
4       Which is better, a laptop or a desktop?   
5       Which is better, a laptop or a desktop?   

                                          query         tag  \
0  What is the difference between sex and love?    original   
1               difference between sex and love  annotation   
2   Which is the highest mountain in the world?    original   
3                        highest mountain world  annotation   
4       Which is better, a laptop or a desktop?    original   
5                      better laptop or desktop  annotation   

                                            response  
0  {'results': [{'trec_id': 'clueweb12-1214wb-88-...  
1  {'results': [{'trec_id': 'clueweb12-1811wb-62